In [1]:
import os
import csv
import platform
import pandas as pd
import networkx as nx
from graph_partitioning import GraphPartitioning, utils

run_metrics = True

cols = ["WASTE", "CUT RATIO", "EDGES CUT", "TOTAL COMM VOLUME", "Qds", "CONDUCTANCE", "MAXPERM", "NMI", "FSCORE", "FSCORE RELABEL IMPROVEMENT", "LONELINESS"]

pwd = %pwd

config = {
    "DATA_FILENAME": os.path.join(pwd, "data", "predition_model_tests", "network", "rand_edge_weights", "network_1.txt"),
    #"DATA_FILENAME": os.path.join(pwd, "data", "predition_model_tests", "network", "network_1.txt"),
    "OUTPUT_DIRECTORY": os.path.join(pwd, "output"),

    # Set which algorithm is run for the PREDICTION MODEL.
    # Either: 'FENNEL' or 'SCOTCH'
    "PREDICTION_MODEL_ALGORITHM": "PATOH",

    # Alternativly, read input file for prediction model.
    # Set to empty to generate prediction model using algorithm value above.
    "PREDICTION_MODEL": "",

    "PARTITIONER_ALGORITHM": "PATOH",

    # File containing simulated arrivals. This is used in simulating nodes
    # arriving at the shelter. Nodes represented by line number; value of
    # 1 represents a node as arrived; value of 0 represents the node as not
    # arrived or needing a shelter.
    "SIMULATED_ARRIVAL_FILE": os.path.join(pwd,
                                           "data",
                                           "predition_model_tests",
                                           "dataset_1_shift_rotate",
                                           "simulated_arrival_list",
                                           "percentage_of_prediction_correct_100",
                                           "arrival_100_1.txt"
                                          ),
    
    # File containing the prediction of a node arriving. This is different to the
    # simulated arrivals, the values in this file are known before the disaster.
    "PREDICTION_LIST_FILE": os.path.join(pwd,
                                         "data",
                                         "predition_model_tests",
                                         "dataset_1_shift_rotate",
                                         "prediction_list",
                                         "prediction_1.txt"
                                        ),

    # File containing the geographic location of each node, in "x,y" format.
    "POPULATION_LOCATION_FILE": os.path.join(pwd,
                                             "data",
                                             "predition_model_tests",
                                             "coordinates",
                                             "coordinates_1.txt"
                                            ),

    # Number of shelters
    "num_partitions": 4,

    # The number of iterations when making prediction model
    "num_iterations": 1,

    # Percentage of prediction model to use before discarding
    # When set to 0, prediction model is discarded, useful for one-shot
    "prediction_model_cut_off": 1.0,

    # Alpha value used in one-shot (when restream_batches set to 1)
    "one_shot_alpha": 0.5,

    # Number of arrivals to batch before recalculating alpha and restreaming.
    # When set to 1, one-shot is used with alpha value from above
    "restream_batches": 1000,

    # When the batch size is reached: if set to True, each node is assigned
    # individually as first in first out. If set to False, the entire batch
    # is processed and empty before working on the next batch.
    "sliding_window": False,

    # Create virtual nodes based on prediction model
    "use_virtual_nodes": False,

    # Virtual nodes: edge weight
    "virtual_edge_weight": 1.0,

    # Loneliness score parameter. Used when scoring a partition by how many
    # lonely nodes exist.
    "loneliness_score_param": 1.2,

    ####
    # GRAPH MODIFICATION FUNCTIONS

    # Also enables the edge calculation function.
    "graph_modification_functions": True,

    # If set, the node weight is set to 100 if the node arrives at the shelter,
    # otherwise the node is removed from the graph.
    "alter_arrived_node_weight_to_100": False,

    # Uses generalized additive models from R to generate prediction of nodes not
    # arrived. This sets the node weight on unarrived nodes the the prediction
    # given by a GAM.
    # Needs POPULATION_LOCATION_FILE to be set.
    "alter_node_weight_to_gam_prediction": False,

    # Enables edge expansion when graph_modification_functions is set to true
    "edge_expansion_enabled": True,

    # The value of 'k' used in the GAM will be the number of nodes arrived until
    # it reaches this max value.
    "gam_k_value": 100,

    # Alter the edge weight for nodes that haven't arrived. This is a way to
    # de-emphasise the prediction model for the unknown nodes.
    "prediction_model_emphasis": 1.0,
    
    # This applies the prediction_list_file node weights onto the nodes in the graph
    # when the prediction model is being computed and then removes the weights
    # for the cutoff and batch arrival modes
    "apply_prediction_model_weights": True,

    "SCOTCH_LIB_PATH": os.path.join(pwd, "libs/scotch/macOS/libscotch.dylib")
    if 'Darwin' in platform.system()
    else "/usr/local/lib/libscotch.so",
    
    # Path to the PaToH shared library
    "PATOH_LIB_PATH": os.path.join(pwd, "libs/patoh/lib/macOS/libpatoh.dylib")
    if 'Darwin' in platform.system()
    else os.path.join(pwd, "libs/patoh/lib/linux/libpatoh.so"),
    
    "PATOH_ITERATIONS": 10,
        
    # Expansion modes: 'no_expansion', 'avg_node_weight', 'total_node_weight', 'smallest_node_weight'
    # 'largest_node_weight', 'product_node_weight'
    # add '_squared' or '_sqrt' at the end of any of the above for ^2 or sqrt(weight)
    # add '_complete' for applying the complete algorithm
    #    for hyperedge with weights: A, B, C, D
    #      new weights are computed
    #       (A*B)^2 = H0
    #       (A*C)^2 = H1, ... Hn-1
    #      then normal hyperedge expansion computed on H0...Hn-1
    # i.e. 'avg_node_weight_squared
    "PATOH_HYPEREDGE_EXPANSION_MODE": 'total_node_weight_sqrt_complete',

    # Alters how much information to print. Keep it at 1 for this notebook.
    # 0 - will print nothing, useful for batch operations.
    # 1 - prints basic information on assignments and operations.
    # 2 - prints more information as it batches arrivals.
    "verbose": 1
}

#gp = GraphPartitioning(config)

# Optional: shuffle the order of nodes arriving
# Arrival order should not be shuffled if using GAM to alter node weights
#random.shuffle(gp.arrival_order)

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import scipy

iterations = 1000
#modes = ['product_node_weight_complete_sqrt']
modes = ['no_expansion', 'avg_node_weight_complete', 'total_node_weight_complete', 'smallest_node_weight_complete','largest_node_weight_complete']
#modes = ['no_expansion']

for mode in modes:

    metricsDataPrediction = []
    metricsDataAssign = []
    
    dataQdsOv = []
    dataCondOv = []

    config['PATOH_HYPEREDGE_EXPANSION_MODE'] = mode
    print('Mode', mode)
    for i in range(0, iterations):
        if (i % 50) == 0:
            print('Mode', mode, 'Iteration', str(i))
            
        config["DATA_FILENAME"] =  os.path.join(pwd, "data", "predition_model_tests", "network", "network_" + str(i + 1) + ".txt")
        
        gp = GraphPartitioning(config)
        gp.verbose = 0
        gp.load_network()
        gp.init_partitioner()

        m = gp.prediction_model()
        metricsDataPrediction.append(m[0])

        '''

        #write_graph_files
        #
        gp.metrics_timestamp = datetime.datetime.now().strftime('%H%M%S')
        f,_ = os.path.splitext(os.path.basename(gp.DATA_FILENAME))
        gp.metrics_filename = f + "-" + gp.metrics_timestamp


        if not os.path.exists(gp.OUTPUT_DIRECTORY):
            os.makedirs(gp.OUTPUT_DIRECTORY)
        if not os.path.exists(os.path.join(gp.OUTPUT_DIRECTORY, 'oslom')):
            os.makedirs(os.path.join(gp.OUTPUT_DIRECTORY, 'oslom'))

        
        file_oslom = os.path.join(gp.OUTPUT_DIRECTORY, 'oslom', "{}-all".format(gp.metrics_filename) + '-edges-oslom.txt')
        with open(file_oslom, "w") as outf:
            for e in gp.G.edges_iter(data=True):
                outf.write("{}\t{}\t{}\n".format(e[0], e[1], e[2]["weight"]))
        
        #file_oslom = utils.write_graph_files(gp.OUTPUT_DIRECTORY,
        #                             "{}-all".format(gp.metrics_filename),
        #                             gp.G,
        #                             quiet=True)

        community_metrics = utils.run_community_metrics(gp.OUTPUT_DIRECTORY,
                                                        "{}-all".format(gp.metrics_filename),
                                                        file_oslom)
        
        dataQdsOv.append(float(community_metrics['Qds']))
        dataCondOv.append(float(community_metrics['conductance']))
        '''


    ec = ''
    tcv = ''
    qds = ''
    conductance = ''
    maxperm = ''
    nmi = ''
    lonliness = ''
        
    qdsOv = ''
    condOv = ''

    dataEC = []
    dataTCV = [] 
    dataQDS = []
    dataCOND = []
    dataMAXPERM = []
    dataNMI = []
    dataLonliness = []

    
    for i in range(0, iterations):
        dataEC.append(metricsDataPrediction[i][2])
        dataTCV.append(metricsDataPrediction[i][3])
        dataQDS.append(metricsDataPrediction[i][4])
        dataCOND.append(metricsDataPrediction[i][5])
        dataMAXPERM.append(metricsDataPrediction[i][6])
        dataNMI.append(metricsDataPrediction[i][7])        
        dataLonliness.append(metricsDataPrediction[i][10])


        # UNCOMMENT FOR BATCH ARRIVAL
        #dataECB.append(metricsDataAssign[i][2])
        #dataTCVB.append(metricsDataAssign[i][3])

        if(len(ec)):
            ec = ec + ','
        ec = ec + str(metricsDataPrediction[i][2])
        if(len(tcv)):
            tcv = tcv + ','
        tcv = tcv + str(metricsDataPrediction[i][3])

        if(len(qds)):
            qds = qds + ','
        qds = qds + str(metricsDataPrediction[i][4])

        if(len(conductance)):
            conductance = conductance + ','
        conductance = conductance + str(metricsDataPrediction[i][5])

        if(len(maxperm)):
            maxperm = maxperm + ','
        maxperm = maxperm + str(metricsDataPrediction[i][6])

        if(len(nmi)):
            nmi = nmi + ','
        nmi = nmi + str(metricsDataPrediction[i][7])

        if(len(lonliness)):
            lonliness = lonliness + ','
        lonliness = lonliness + str(dataLonliness[i])
        
        '''
        if(len(qdsOv)):
            qdsOv = qdsOv + ','
        qdsOv = qdsOv + str(dataQdsOv[i])

        if(len(condOv)):
            condOv = condOv + ','
        condOv = condOv + str(dataCondOv[i])
        '''

    ec = 'EC_PM,' + config['PATOH_HYPEREDGE_EXPANSION_MODE'] + ',' + str(scipy.mean(dataEC)) + ',' + str(scipy.std(dataEC)) + ',' + ec
    tcv = 'TCV_PM,' + config['PATOH_HYPEREDGE_EXPANSION_MODE'] + ',' + str(scipy.mean(dataTCV)) + ',' + str(scipy.std(dataTCV)) + ',' + tcv

    lonliness = "LONELINESS," + config['PATOH_HYPEREDGE_EXPANSION_MODE'] + ',' + str(scipy.mean(dataLonliness)) + ',' + str(scipy.std(dataLonliness)) + ',' + lonliness
    
    qds = 'QDS_PM,' + config['PATOH_HYPEREDGE_EXPANSION_MODE'] + ',' + str(scipy.mean(dataQDS)) + ',' + str(scipy.std(dataQDS)) + ',' + qds
    conductance = 'CONDUCTANCE_PM,' + config['PATOH_HYPEREDGE_EXPANSION_MODE'] + ',' + str(scipy.mean(dataCOND)) + ',' + str(scipy.std(dataCOND)) + ',' + conductance
    maxperm = 'MAXPERM_PM,' + config['PATOH_HYPEREDGE_EXPANSION_MODE'] + ',' + str(scipy.mean(dataMAXPERM)) + ',' + str(scipy.std(dataMAXPERM)) + ',' + maxperm
    nmi = 'NMI_PM,' + config['PATOH_HYPEREDGE_EXPANSION_MODE'] + ',' + str(scipy.mean(dataNMI)) + ',' + str(scipy.std(dataNMI)) + ',' + nmi

    #qdsOv = 'QDS_OV,' + config['PATOH_HYPEREDGE_EXPANSION_MODE'] + ',' + str(scipy.mean(dataQdsOv)) + ',' + str(scipy.std(dataQdsOv)) + qdsOv
    #condOv = 'CONDUCTANCE_OV,' + config['PATOH_HYPEREDGE_EXPANSION_MODE'] + ',' + str(scipy.mean(dataCondOv)) + ',' + str(scipy.std(dataCondOv)) + condOv

    print(ec)
    print(tcv)
    print(lonliness)
    print(qds)
    print(conductance)
    print(maxperm)
    #print(qdsOv)
    #print(condOv)

Mode no_expansion
Mode no_expansion Iteration 0
Mode no_expansion Iteration 50
Mode no_expansion Iteration 100
Mode no_expansion Iteration 150
Mode no_expansion Iteration 200
Mode no_expansion Iteration 250
Mode no_expansion Iteration 300
Mode no_expansion Iteration 350
Mode no_expansion Iteration 400
Mode no_expansion Iteration 450
Mode no_expansion Iteration 500
Mode no_expansion Iteration 550
Mode no_expansion Iteration 600
Mode no_expansion Iteration 650
Mode no_expansion Iteration 700
Mode no_expansion Iteration 750
Mode no_expansion Iteration 800
Mode no_expansion Iteration 850
Mode no_expansion Iteration 900
Mode no_expansion Iteration 950
EC_PM,no_expansion,109.441,14.2816147196,138,113,94,105,131,97,101,86,110,112,91,114,96,118,132,83,99,110,114,125,97,98,105,128,125,110,102,125,127,121,135,114,108,119,113,103,103,112,107,92,109,118,115,93,86,101,85,114,115,103,114,101,122,116,110,156,89,119,116,93,95,95,100,105,120,107,104,113,86,86,132,102,134,103,123,109,86,96,90,111,99,101